In [1]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

In [2]:
iris = load_iris()
X_train, y_train = StandardScaler().fit_transform(iris.data), iris.target

In [3]:
X_train.shape, y_train.shape

((150, 4), (150,))

In [4]:
def performance(model):
    score = cross_val_score(model, X_train, y_train, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=0), scoring='accuracy').mean()
    return score

In [5]:
class MyDecisionTree(BaseEstimator):
    def __init__(self, max_depth=10):
        self.max_depth = max_depth
        self.tree_ = None
        
    def calculate_gini(self, y):
        n = len(y)
        y_distribution = {}
        for member in y:
            y_distribution[member] = y_distribution.get(member, 0) + 1
        gini = 1
        for num in y_distribution.values():
            gini -= (num / n) ** 2
        return gini

    def split_data(self, X, y, feature_index, split_point):
        left_index = (X[:, feature_index]<=split_point)
        right_index = (X[:, feature_index]>split_point)
        return X[left_index], X[right_index], y[left_index], y[right_index]

    def select_feature(self, X, y, n, m):
        best_info_gain = [0] * m
        best_split_point = [None] * m
        base_gini = self.calculate_gini(y)
        for i in range(m):
            this_best_info_gain = 0
            this_best_split_point = 1
            for j in range(n):
                split_point = X[j, i]
                _, _, y_left, y_right = self.split_data(X, y, i, split_point)
                this_gini = len(y_left) / n * self.calculate_gini(y_left) + len(y_right) / n * self.calculate_gini(y_right)
                this_info_gain = base_gini - this_gini
                if this_info_gain > this_best_info_gain:
                    this_best_info_gain = this_info_gain
                    this_best_split_point = split_point
            best_info_gain[i] = this_best_info_gain
            best_split_point[i] = this_best_split_point
        selected_feature_index = np.argmax(best_info_gain)
        return selected_feature_index, best_split_point[selected_feature_index]

    def major(self, y):
        y_distribution = {}
        for member in y:
            y_distribution[member] = y_distribution.get(member, 0) + 1
        y_members = list(y_distribution.keys())
        y_numbers = list(y_distribution.values())
        major_member = y_members[np.argmax(y_numbers)]
        return major_member

    def build_tree(self, X, y, depth=1):
        n = X.shape[0]
        m = X.shape[1]
        tree = {}
        if len(set(y)) == 1:
            return y[0]
        if depth >= self.max_depth:
            return self.major(y)
        depth += 1
        feature_index, split_point = self.select_feature(X, y, n, m)
        X_left, X_right, y_left, y_right = self.split_data(X, y, feature_index, split_point)
        tree[(feature_index, split_point, True)] = self.build_tree(X_left, y_left, depth)
        tree[(feature_index, split_point, False)] = self.build_tree(X_right, y_right, depth)
        return tree
    
    def go_down_the_tree(self, tree, x):
        if np.isscalar(tree):
            return tree
        index, split_point, _ = list(tree.keys())[0]
        if x[index] <= split_point:
            return self.go_down_the_tree(tree[(index, split_point, True)], x)
        else:
            return self.go_down_the_tree(tree[(index, split_point, False)], x)

    def fit(self, X, y):
        self.tree_ = self.build_tree(X, y)
        return self
    
    def predict(self, X):
        y_pred = []
        for x in X:
            y_pred.append(self.go_down_the_tree(self.tree_, x))
        return np.array(y_pred)

In [6]:
performance(MyDecisionTree(max_depth=4))

0.9533333333333334